In [4]:
#run again ater running next cell
import dagshub
import mlflow
import sys
import torch
import json
import numpy as np
import requests
from pathlib import Path
from pprint import pprint
from genre_classifier.model import MFCCDataModule

In [2]:
path = Path.cwd().parent /"src" 
sys.path.append(str(path))

In [6]:
dataset_path = Path.cwd().parent / "data" / "processed" / "genres_mfccs.json"

In [ ]:
dagshub.init(
    repo_owner="stephenjera",
    repo_name="Genre-Classification",
    mlflow=True,
)

In [7]:
mlflow.set_tracking_uri('http://localhost:5000')

In [8]:
# Load one sample from data 
X, y, mappings = MFCCDataModule.load_data(dataset_path)

Loading Data


In [9]:
reverse_dict = {v: k for k, v in mappings.items()}

In [10]:
X = torch.tensor(X, dtype=torch.float32).clone().detach()

## Predict from deployed model

run in command line
```shell
mlflow models serve -m "models:/genre-classifier/19" --port 1234 --no-conda
```


In [ ]:
print(X.shape)
print(X[:1].shape)
print(X[0].shape)

In [ ]:
#X = np.array(X, dtype=np.float128)
#X = torch.tensor(X, dtype=torch.int64).clone().detach()
single_mfcc = X[:1].tolist()
#single_mfcc = single_mfcc.astype(np.float64).tolist()
#single_mfcc = torch.from_numpy(X[:1]).float().tolist()

# Make prediction request
url = 'http://localhost:1234/invocations'
headers = {'Content-Type': 'application/json'}
data = {
  "instances": single_mfcc
}
response = requests.post(url, headers=headers, data=json.dumps(data))

# Print prediction
pprint(response.json())

In [ ]:
predictions = response.json()['predictions']

# Convert the predictions to a numpy array
predictions_array = np.array(predictions)

# Find the index of the maximum value
argmax_index = np.argmax(predictions_array)

print(f"prediction:{argmax_index}, {reverse_dict[argmax_index]} Actual {y[0]}, {reverse_dict[y[0]]}")

## Predict from loaded model

In [12]:
model_uri = "models:/genre-classifier/1"
loaded_model = mlflow.pytorch.load_model(model_uri)

In [13]:
prediction = loaded_model.predict_step(torch.tensor(X[:2]))

/var/folders/hh/nxv389h50tjc59gglqdnd7_m0000gn/T/ipykernel_30725/2340729352.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prediction = loaded_model.predict_step(torch.tensor(X[:2]))


In [14]:
# Get the predicted class
predicted_class_index = prediction.argmax().item()
print(f"The predicted class index is {predicted_class_index}")

The predicted class index is 12


In [15]:
print(
    f"prediction:{predicted_class_index}, {reverse_dict[predicted_class_index]} Actual {y[1]}, {reverse_dict[y[1]]}"
)

KeyError: 12